# GENERAL NOTES
Is ml even needed? What ie the right solution?How much compute do we have? How fast do we need answers? How many users do we serve? How will this change across time? Is there any offline capabilites? What are our ethical responsibilities (data privacy, Biases, etc). Also, keep in mind, you never know parameters, you only know estimations of parameters

## When in doubt, ask "what are we doing this for; what is the objective, and what are the constraints? What are other ways about it?"

## When in doubt, approximate!

## When in doubt, say the obvious things! Say all of the obvious things!


# ML Sys Design Overview




## Design Framework


### Initial Questions
- Why do we want to do this? Who will be consuming this and how? What metrics wdo we expect to move?
- What is the size of the dataset? What are the latency requirmenets?
- What will the inputs and outputs be? What is the high level flow?

### High Level Flow
#### Rec Sys Example Flow
- candidate fetch stage
- filters and other business logic
- pre-ranking stage
- full ranking stage
- reranking stage

#### Inf Extraction Example Flow
- content classification
- entity extraction
- entity resolution

### Data Collection Processing and Storing
- What kind of data do we need? How much?
- Where does it come from? How expensive is it?
- How do we sampel from the data? How do we remove outliers, annotate, and account for collection / labeling biases?

### Feature engineering
- What features are important, and how do we know?
- How can we leverage the features to optimize the model?

### Modeling and Evaluation
- Evaluating and training different model architectures
- Dataset quality and bias issues
- Over and underfitting, hyperparameter tuning
- how do you balance offline goals (loss function minimization) and online goals (maximizing revenue / user engagement)

### Deployment and serving
- How much complexity to the system is added by the model?
- How do we monitor and maintain the model?
- How do we modify models once they are deployed into production?
- How do we deal with perpetuating biases in deployed models? 

# 1:) Problem Clarification

## 1.) Why would we want to recommend content to the user?
- Relevance | Discoverability | Increased engagement | Increased revenue | Better Understanding of Customer Behavior
- N.b. 80% of Netflix watches come from their recommendations
  
## 2.) What metrics do we expect to improve?
- User retention
- Positive and negative engagements, e.g. clicks and reports
- User "state progression" -> convert light users to medium and heavy users
- Subsriptions / follows - increase density of network, benefical all the way around
- impressions, costs per engagement, monetizable engagements.

## 3.) What kinds of content should we recommend?
- Non personalized content - e.g. trending content, new content, highly rated
- Personalized content - in network and out of network
- Diverse conent - avoid a filter bubble
- Serendipity - random / serendipitous recommendations to keep users engaged / refreshed
- Also need to include business considerations - advertiser bids (with potential throttling for burnrate smoothness), boosting, revenuse share, etc.
- Ad confliction / repeated advertisements

## 4.) Blending different content types (e.g. in&out of network, ads)

- Interleaving
- Heuristics
- Separated
- Clustered
- Global ranking
- Reinforcement Learning - develop a frameform for insertion strategy
  

## 5.) Describe operational parameters of this recommender system

- Latency considerations: amount of time for the system to run inference. Typically 100s of ms is the upper limit for real time systems
- Throughput: possible processing volume. Typically 100s of requests per second per million users.
- Number of candidates: starting point for system, Pinterest has a vatalog of billions
- Number of results; output of system, typically one to dozens, depending on the system
- 

# 2:) High Level Design


## 6.) Describe the high level design for this recommender system
- Candidate generation --> How is this actually done?
- Filters - quick assessments
- Pre-ranking  / lightweight ranking
- heavy ranking / full ranking
- reranking )optional) with adtional factors, such as diversity or freshness

## 7.) How do you surmount the cold start problem?

Most recommender systems rely on user-user and user-item engagemetns. Some techniques to bootstrap are:
- make them fill out information e.g. demographics, interests
- heuristics - take the average of embeddings of similar users, or display non personalized content (trending, diverse, etc)
- learning period - increase visibilty of new items / variance, can decay over time
- Contextual bandits - systematic approach. Adjust strategy based on feedback to maximimze long term engagement. Upper Confidence Bound balances explore and exploint by recommending itemst aht have the highest upper confidence bound on reward given the available context.

Some modeling techniques are:
- Fallback models - when there isn'te enough data, provide less acturate features
- Collaborative models - interpolate beteween content-forarrd and conventional modeling. This provides a difficulty in complex formulation of objective funciton
- hybrid models - combine behaviorual and latent contnet.
- Dropout - force the model to apply dropout

# 3:) Data Collection

## 8.) What datasets do we need?

- RecSys typically need user-tiem interaction data (clicks purchaces, follows)
- Demographic features can be helpful
- metadata can be helpful
- positive examples (instances where correct answer is known)
- negative examples (instances where correct answer is not in teh candidate list, i.e. we may not know what the answer is, but we have some examples of what the answer isn't
- Typically user-item interactions are sparce, candidate generators train by way of negative sampling - select small subset of items that are not relevant
- preranking - knowledge distillaiton - inputs to the model, along with outputs, the student model is optimized to produce outputs that are similar to the theacher's outputs. Unlike heavy ranking, don't need ground turth labels. More options for collecting data as a result. One appraoch is to pull all input candidates and ask a heavy ranker to score them offline, thus the pre-ranking model can learn what the heavy ranker might like.
- Counterfactual reasoning framework: estimate the effect of changing one or more variables ina  system. Estimate how a user would have behaved if a different set of candidates were presented, minimze regret
   Traditionally done during A/B experiments, but that can be time and resource intensize. Instead, counteractual reasoning can be employed offline to tackle this. One approach is to allow all generated candidates to survive until heavy ranking and annotate each step where a candidate is scored / filtered out. Mark the candidates ulitimately served. It is possible then ot compute the amount of regret htat each heuristic contriputes. It is not flawless as it only esimates the user engangement, it can help pinpoint frutiful paths. Can be supplemented with A/B tests.
  


## 9.) How do we collect these datasets?

- Data must be representative of the users and items, must be large enough to train a model, and must be constructed in a way that minimizes the impact of bias
### Candidate generation stage: 
- negative sampling is used in place of optimizing against all items or only positive items (whichmay result in overprediciton of negative items)
- Uniform Sampling - randomly select anything not in the users history as negative samples. Simple, but may not force model to distinguish between positive items and negatives that resemble positives
- Popularity based - more popular = more liekly to be positive
- Popularity dampened - reduces advantage of very popular items
- Faraway negative sampling - select negatives that are very far away from positive items, or with lower scores. May produce uninformative neavies similar to random sampling
- Hard Negative Sampling - contemporary approach, selects negatives that are similar to positives. Can be prone to false negatives. PinSange and Monte Carlo Negative Sampling are examples.
- GANs - KBGAN, for example - softmax based model as generator to sample negatives likley to confuse the discriminator, margin loss based model for discriminator. Hard to generate many samples.

### Light Ranking stage
- Biased sampling - Not sure
- Uniform sampling - uniformly random sampling. Most diverse sampling, but unable to evaluate ranking metrics
- Request-level - sample all input candidates - im not quite sure what this means
- In practice some combination is often best; uniform for training, request level for evaluation, or equest can be used to boost diversity, etc.

### Heavy Ranking Stage

- Stream of data from sandidates served to users. Fed back into model to improve itself over time.
- Update frequency - data freshness is important.
- Delayed feedback - in many real time systems, interactions are only available fter a (possibly long and random) delay. This will cause naive stratigies which consider any data point a negative example until a positive label becomes avialabe tend ot underpredict these instances.
- Negative sampling: Number of positive samples typically much much smaller than the pool of served candidates, leavding to severe class imbalance problems. for example, positive ad impressions may be less than 0.01%. This skews towareds negative examples. Oversampling and downsampling are commonly used methods
- Explore/exploit - Needed to aboid biases, as discussed below



## 10.) What sources of biases might be found in the dataset?
- Position bias - items at top are more likely to be clicked by user.
- Presentation of the item influeneuse the users decision. E.g. if ad happens to be next to attractive images, might get unfair number of clicks
- trust bias - users trust the recommneder system too much, don't actually act with own intent
- serving bias - algorithimic bias, recommends similar items to those tha have historically performed will. Perpetuation of biases.



## 11.) How do we mitigate serving bias?
- failure to explore candidate space. Explore/exploit is essential to understand in rec sys.
- Epsilon-greedy - add some random data to the training set. recommeds the item with a prob of 1-epsilon, while uniformly randomly selecting something oelse with probabiliyt of epsilon. Tends ot ouptperform more advanced methods like bandit approaches.
- Contextual bandits. Randomly sample from a posterior distribution
  Causual inference - esimate the causeal effects of alternate actions.
  learning period - boost new items with a decay based on time. GIves higher weights to newer data and gradually decrease weight over time. does not address other problems like lack of diversity of exploration. 


## 12.) How do we mitigate positional bias
- Add randomness to the position of items, e.g. randomly shuffle
- add positional features - incorporate it into the training to understand its effect
- can model positional bias, use it as a normlaizer or regularizer, butassumes that the relationship is known and can be modeled.
- Multi-task model - youtube implements this, at least at one point
- Adversarial training - define an auxillary task that predicts the positon of items in teh training. Negate the gradient to comban influence of position. 

# 4:) Candidate Generation

## 13.) Where do candidates come from? List Potential Sources

### Non-peronalized sources
- popular conent
- trending content
- new conent
- trending in contenxtual areas (categories, gerographic locations, topics)

### Perosnalized in-network
- items generated by users or companies the user is following / connectd to
- historical conent - user has previouly consumed, highly rated, etc.

### Personalized out of network
- Content based filtering - items similar to previously viewed contnet
- Collaborative filtering - users own preferences,etc.
  


## 14.) What are the benefits and drawbacks of each source?

- Non personalized sources - good starting point for new users, good for popular content, but may not be tailerd and might be limited in diversity
- Personalized in network - highly relevant and finds new items, but less diverse and not useful for new users
- Personalize out of network - highly personalized and tailored, also good for new users, but can require significant user data and processing power to generate recommendations based on similarity or content based filtering. May result in filter bubble.

## 15.) Describe how candidate generation works at a high-level

- Fetch sources - get sources that can act as seeds for generating candidates, e.g. embeddings. For example, a music streaming service might fetch sources such as user's listening history, songs on the users playlist, and artists they folow.
- Generate candidates - use a candidate generation algo - graph based algos, DNNs, etc. (see below.)
- Filter candidates - merge and prune to get to a fair number for pre-ranking.
- Candidates can be generated from different sources, making this stage many to many. Twiller applies dozens of combos to generate candidates for thier systems.

## 16.) What are some algorithms for generating candidates?

### Neighborhood Based Methods:
Leverage user preferences between items to generate candidates. Slope One and Pearson R are examples. Slope one is an item-based method that predicts the rating of an item by consdiering the differences in ratings between pairs of items that users have rated. Perarson R is a user based method that computes the corellation coefficent between teh use r and other users to identify similar users and recommend items based on their ratings. These do not learn embeddings.

### Graph-based methods:
Model the relationships between items or users as a graph and use graph algos to generate candidates. See graph embeddings below for more.

### Latent methods
Matrix facotirzation and DNNs to generate candidates. Captures latent relationships that are not directly observable in the data. PinSage is a GCN. Aggregates feature information from a suser's one-hop neighborhood. Google uses Two-Tower.

## 17.) How might you develop an embedding approach to retrieve canddiates?

### Matrix Factorization (MF)
- Decombpose rating matrix into low-rank matrices. Common technique is Singular Value Decomposition. Decomposes originla matrix into three separate matrixces, left, right, and singular value. Left matrix is embeddings.  Another is weighted matrix factorization. Optimization techniques such as SGD and Weighted altenrating least squares can be used to optimize the embeddings. Can be difficult to incorporate user and item features. Can capture up to second order features.

### Deep Neural Networks
Improvement over MF tecniques. Highly nonlinear. Campures complext features more efficently than conventional MF methods. Popular models are sequecne models, graph neural networks, and Two Towers.
#### Sequence Models
Neural Language models. Can be used to create user and item embeddings. Treats users history as a sequence of words. Neural language model learns embeddings that caputre the sequential dpeendinecies. Leverage attention mechanisms in tranformers to generate embeddigns. Learn more accurate embeddings. Air BNB does this. 
#### Graph Neural Netowrks
Graph Convolutional Networks (example) - network desinged to handle graph structured data. Gather and combine feature informaiton from local neighborhoods of nodes by performing convlutions recursiveely. Stack them to caputre both local and global graph structure. LEverages both content and graph structure to generate usre and item embeddings. PinSage and PinnerSage do this. 

### The Two Towers
AKA SplitNet, or vector product models. Divides a DNN into two sub-networks that process user and item features separately. No crossing features between them. Output is a fixed size vector that represents the user and item embeddings, respectively. SCcore can be computed with different similarity functions, such as inner product or hadamard products. Offers an andvantge in that it allows for independent embedding of new users and items. Useful for whern there is a constant influx of new users and items. 

### Graph Embeddings:
Graph embeddings AKA network embeddings, are a tecnique used to convert nodes into compact low dim representaitons. One example is SimClusters - community based graph embedding approach. 

Another exmaple is Heterogeneous Information Networks. 

## 18.) At scale, candidate generation cannot score every potential candidate in real-time. What are some ways to solve this?

- Use Approximate Nearest Neighbors (ANN) to efficenlty find top candidates. Efficently search high dim spaces by approximating the distnaces betweenpoints. Methods include:
  - Locality Sensitive Hasing
  - Inersted File Index sith Product Quantization - Compress and quantie data into compact codes
  - Hierarchical Navigabele Small WOrld - Construct a graph of high dim items with high clustering coeff - nearby nodes tend to be connected.
  - Score offline - precompute and store a list of top candidates for each user or item, can be effective for related-time recommendations

## 19.) How do you index newly created content?

- In Matrix Factorization,  just solve the matrix equation (Weighted alternating least squares)
- In Two Tower approach - save resulting embedding
- Sim clusters - straightforard

## 20.) Suppose we need to limit the number of candidates. How should we merge and prune from differtn algorithms?
Several otpions are available:
- Heuristics - known performance heursitics, proor knowledge. Might not take into account qaulity of candidates
- Thresholding - only let over a scrore through. Does not take into account relative qulaty
- Universal scoring - normalze scores, but can create addtional work and might be a misalighmnet of objectives
- regression - linear regression, this can cause latency issues.
- Light ranking - very fast pre ranker - use more comlex model, again there are downsides

## 21.) Why not use the candidate generator to rank items?

- Multiple candidate generators may be used in a system, scores may not be directly comparable.
- Ranking task involves considering smaller pool of candidates, lead to more accurate
- Scoring function may not be optimized for ranking, i..e ranking and candidate generation are not synonyms. Multiple objectives might be needed for ranking. Allows for separation of objectives.

# 5.) Light Ranking

## 22.) What is pre-ranking? What does it do?

A light ranking model is a simpliefied version of a heavy randking model. Quickly rands candidates, screens for top candidates. Optimizes recall or cumulative gain. Employed when latency of rec sys is critical / important. Typically eliminates 25 to 90 percent of candidates. Allows for the heavy ranker to be more complex. 




## 23.) What should the pre-ranking model learn?

KNOWLEDGE DISTILLATION - a technique used to transfer knowledg from a large, complex model to a msller, simplier model. Train the studen tmodel to mimic th eouptus of hte teacher model. 

KD involvees using outputs of hte teachrer model as targets for the stduent, minimze the difference between its own outputs and target outputs. Main drawback is that if the teacher model is not well trained the distilled knowlege will be wrong too.

Can also learn directly from groutnd truth labels, but this might be a bad idea - getting tground trught may be impractical, training data might be imblanaaced, correlation can be enefical for reducing system error. However can incororpate gorund truth into featuers


## 24.) What are some evaluation metrics for the model?

- Normalized cumulative gain.
- top-k recall
- expected top-k loss - differnece in gain value for top-k candidates

## 25.) What are some suitable algorithms for this model?

- Heuristics - averageing predicted engagements
- Lgistic regression, quick to train and update
- Two Tower arcitectures - good but misses crossing features
- Two Tower to Lightweight DNNs

## 26.) Suppose we need to pre-rank a large number of candidates. How do we optimize the model for this?

Cached or precompute embeddings
Parallelism, adjust atch size
Densify input features / compress them
cache at multiple levels to minimize feature hydration
opmitize architecture - reduce size and complexity- apply sqeezing
Compving two tower with above concetps
Cace pairwise model scores, precomute all offline. Can incur storage costs. Not a good idea if data distributions shift

# 6.) Feature Engineering


## 27.) What features should we use? What are their intuitions?

### Target Features
Describe the user
- demographics
- relevant keywords
- User's interests
- Request-level features (spefici page of the the user interface, origiginating query, device, client, carrier, network condition, geogrpahic location, time of daty, day of week, etc.
- How much content they consume
- Embedding features - users' engagement history, embeddings of subsribers, etc.
- revenue (ad) models - average bid depth, bid value, click through rate, fill rate, network valuation

### Candidate Features
- Static attributes - ID, category, etc
- numeric features, history, recent engagements
- content's author, advertiser, ad campaighn
- metadata - type of ad product, bit type, currency
  
### Crossing features
How much content has by this author has the user consumed? 
When was the last time they consumed it?
Explicti corssing - use din logistic regression
Embedding crossing - crossing the embeddings
Deep crossings (deep and cross network rom google)


## 28.) How do we handle textual or id-based features?
- Low cardinality, can use one-hot encoding. Rule of thumb, what less than 20% of the dataset size or squar root fo the dataset size for cardinality. 
- high cardinality - hashing - reduce the dimensionality of the input vector.
- Embeddings - repsresent categorical features as contionous valued vectors in lower dim space. However, may over generalize.

## 29.) What about counting features? What are problems with using counting features?
Couting features can get large. Good idea to normalize in some way. 
- Log transforms - log plus 1, good idea but doesnt handle negative values
- z-score scaling (quantile distributions) can be computationally expensive
- Bucketing - binning / discretizing, quantile bountds. Clipping - cut off above and below thresholds, good for outliers.
- MLP directly and using the oupt as input to remaining layers.

# 7.) Modeling & Evaluation

## 30.) What should the heavy ranking model learn?

Learning to Rank is the process of training the ranking model using data.
- Pointwise - predict a single relevance score or each candidate. Sigmoid cross entropy loss is pospular. Disadvanctage of not capuring relationships
- Pairwise - distingiish between preferred and non-preffered candidates. Compare 2 candidates at a time and predict which one would be user preffered. Some times oupterforms pointwise models
- Listwise - take into account entire list. Probabilist loss function that optimizes KL divergence between predicted ranking and ideal ranking. ListMLE maximizes the probaility of the ideal ranking. Downside is significant amount of training data and comp. resourrces are needed.

## 31.) What are some reasonable modeling algorithms?
- Logistic regression. Can be updated through contionus learning
- Gradient boosting - XGBoost trees, caputre non linaer feature interacitons nadhandle missing values.
- Hybrid - Start with a tree, feed into linear classifier. 
DNNsProcess vast amounts of data without perfomance saturation, can be fit to specific problem domains. 

## 32.) Describe what your model architecture might look like.
- Logistic regression
- - DNNs - improve genearalization capability by learning low dim dense embedding, but struggles to learn reprsentation sin spare interactions
  - Google introduced Wdie and Deep. Pointrise ranking models
    , leading to hybrid models
- More recelntly have been using multi ltask learning approaches. Optimize both clicks and likes.
- Attention / transformer based models have emerged as powerful approaches
- Attention mechanisms have three elements:key, query, and value
- key is an input element (current user interaction)
- query is an element we want to focus on)
- vaue is information we want to extract for the query element
- Compute simlairty score, using dot product. Passes to softmax, indicaing releance of query. Sum up weighted value, passed to feedforward network.
- Multiheaded attention, multipe sets of all of these.
  


## 33.) What if the model predictions need to be calibrated?

- ad platforms might charge advertisers for the number off times their ads are dispalyed. As such, there might be a mismath between the billing method use by the ad platform and the avertisers objectives. Leads to situations where an advertiser is paying for impressions that do not lead to the desired conversion event.
- Essential to have calibrated predictions.
Guarantees provided by cross entropy loss are limited to data distributions

Calibration layer may be applied - Platt scaling or isotonic regression. 

## 34.) What are some appropriate evaluation metrics and why?
### Pointwise Metrics
- Area under the precion recall curve
- Average and sqaured error
- Relative Cross entropy

### Ranking metrics
- normoalized discounted cumulative gain - normlaized by a reference score
- Recall at K - measures proportion of relevant titmes that are recommended
- Mean average precision - average precison of recommeded items
- Mean recipriocal rank. Evaluate ability of model to rank rleevant items hgiher than irrelevant items
- ordered pair accuracy measures the proporiton of correctly ordered candidate pairs


### Infrastructure metrics
- Latency
- Success rate
- COmputational Cost
### Product Metrics
- positive engagemetns
- Negative egngagements
- Retetntion
- Revenue
- Conversion rate
- Advertiser cost

## 35.) How do you balance multiple objectives, e.g. Likes vs. Subscribes?

0 train a multi task model. One example is mulit-gate mixture of experts architetcutre. Bottom layer schared across all experts. 
Train separate models 
learn from weighted labels

## 36.) Should we build one multi-task model or combine multiple smaller models?

multi task moels can share knolege
separate models can more easily see per-objective perormance
MTM can see relationships between objectivs
Flexibitlty separate models wins
Complexity - all depends
Reduncdancy - multi task models
Deployment - single models easier to deply serve and maintain
resource constraints - all depends



# 8.) Deployment & Serving

## 37.) How do you enable the model to serve real time requests?
- Loading models
- Serving model requests
- Switching between different versions of the modle (hotswapping)
- management interface
- Hydrate and cace features and predicitons, convert features to appropriate foramt, handle request batching and fialures, and scribe requests.

## 38.) What can we cache in the serving system and where?

-  cache top k items for each user
-  - cache embeddings - low dim vectors that represent users and items
   - cache features - more likely to be reused between calls, can be cached at mutliple levels
   - cache prediction scores. Avoid need to recalculate, reduces server load

## 39.) How often do we update the model?
DEPENDS. Answer below to understand:
- Concept drift - if model relies heavily on id based features or keyword based features, might neeed to be updated more frequenlcy
- covariate drift - as input data changes, might need to change model
- lable drift changes to the target variable
- system drift changes in candidate generation or other upstream, need to retrain model

## 40.) How does online experimentation (A/B) testing work?

- User-binned - users are assigned to two or more groups, groups are compared
- cluster binned - requests are randomly assigned, clusters are compared. Allows for improved moniotirn, including factors such as latencly.
- BEWARE INTERFERENCE -  some ways to mitigate this:
 - sequential experimentaltion - dont run concurrently
 - merge exepriments
 - mutual exclusion - orthogonality
 - multivariate testing. but challenging

## 41.) How should the server support model experimentation? 

- load from a repo where various versions are stored
- rolling deployment, restart the server with the models to be tested
- hotswapping - model snapshots can be swapped in and out of production without need for full restart. May need more sophisticated config setup


## 42.) Describe some ideas for model experiments

Objective - ranking models cna be trainied to optimize different objectives - multi task, compositte, transform
Features - experiemnt with different features
Preprocessing
Algorithims
Model architetcture
Training Dataset
Retraining
Loss functions
Activation functions
regularization methods
hyperparameters



## 43.) How would you go about predicting which offline evaluation metric improves online metrics? 

- Run A/B testing and measure correlation
- Select something like PR AUC, nDCG, or Ordered pair Accuracy, for example. modify the rec system. Observe effect.  

## 44.) How do you debug if online experiment performance drops over time?

- Ensure metrics are statistically significant
- Veryify experiment's healht
- Verify data
- Look for changes in recent deployments
- Check for interference from other experiments
- Check secondary metrics
- Changes in user behavior - could explain issues
- World events (no one wants to watch videos about vaccines after covid)

# ML DESIGN 2

Information retrieval two stage approach: candidate generation and ranking. Here are examples:
- search problems - find relevant results for a query
- finding similar entities - similar to collaborative filtering approaches
- Identifying near duplicates - similar to content based filtering approaches
- entity resolution - maps different representations of an entity to a common taxonomy  -  can cast the problem a s a search problem
- Question answering - answe an NL question by retreving relevant information from a large corpus of documents. Traditionally:
- - Query decomp - reformat querty into smaller units
  - hypothesis generation - geenrate a set of candidate answers
  - Evidence retrieval - retrieve and rank evidence
  - synthesis - generate a coherent and NL answer to the question - needs additional processing such as NL generation
  - answer ranking - rank baseed on relevancy, etc. Similar to heavy ranking

More design questions:
- Classification - supervised learning where goal is to predict a categorical label
- regression - predict a continuous numerical value for a given input
- information extraction - subfield of NLP that involves automatically extracting structred informaiton from unstructed text data.
  clustering - unsupervised learning where you group similar items in a dataset. Doesn't involve predicting a label, instead the algorthm tries to find natural groupings.
  generative modeling - create new data that is similar to the training data. More recently NLG has become popular (e.g. Chat GPT)
  

# 1:) High-Level Design




## 1.) Develop a ML system to extract info from financial reports

Think of systems as many component boxes

ALEXA:

text to speech <-- information extraction / named entity recognition  <-- intent classification <-- automatic speech recognition
more components, more problems

web crawler to get documents --> classifer to check --> ocr to read reports --> translator to get english --> infomration extraction --> entity resolution

## 2.) Design a ML system that can gauge the positivity or negativity of a given text

Sentiment analysis challenges:
- negation: I did not like this
- sarcasm: the weather is nice for a penguin
- comparisons: this move is better than the book, but the book sucks
- inversions: the phone is pretty, but thats it
- negatives that are positive "lebron is a bad man!"

Solutions:
- rule based NLP. easy to implmeent but time consuming and hard to generalize
- lexicon based - fast but hard to generlaize. Weight words, add up the scores
- non neural models - support vecotr machines etc. Can identify sentiment without rules, generlizes well but stil hard to ge tsarcasm, needs alot of labeled data
- Neural Netowrks - context and relationships, LSTMs and transfomer encoders
- RoBERTa is good. applies masking. Can be computationally expensive. 

## 3.) Build a ML system that can identify the prevalence of topics within a news article

- word based: use word embeddings, fast but doesn't captuer relationships. Could extract key word, then embed words, then find similiarity, report most similar words.
- matrix factorization: Latent Semantic Analysis. Non-negative Matrix Factorization.
- probalistic topic modeling: use bayesian inference, assume each document is generated by a set of topic probabiliteis from a dirichlet distribution, then a set of words is drawn from the topics. Then infer the most likely topic distribution for each document of a number of iterations. Hirarchical version is an extension that allows for an unbounded number of topics to be interred. 
- neural topic models: Vareational autoencoders. map to latent space, etc. 

- Vareational autoencoder - words are converted to embeddings, feed forward, reparameteriztion, softmax. 

## 4.) Design a ML system that condenses a body of text into a single paragraph

Two main approaches: extractive and abstractive
- Extractive: select important sentences and arrange them in coherent summary. Readibilty can be a problem.
- - feature extraction
  - sentence selection
  - reduncandy removal
  - sentence ordering
  - coreference resolution
  - summary
- Abstractive
  - encoder-decoder networks trained on large datasets
  - transformers are vetter versions of this. large capaicty. mixture of latent concepts with great precsion
  - can generate coherent and conscise by generating new sentences
  - relies on generating new sentences, can hallucinate


## 5.) Design a ML system that understands commands like "Play the top songs byt he beatles"

Needs:
- Natural Language Understanding
- Intent classification - understaing what hte user wants. "play music". Use a model to classify intput to predefined output. Rule based system. 
- information extraction - extract relevant informaiton. "top songs" artist = "beatles" . Apply named entity recognition techniques
- entity resolution - maps the extracted entites to a specific reepresentation of the system. we need to understand that top songs refers to teh most listed songs and map the beatles to an atrist. Can use a knowledge graph and a mustic api library. 
- Recenly, LLMs smash this up. They can perform intent classification and information extraction in a single step through fine tuning approaches. 

# 2:) Data Collection




## old.) How do you collect data and prepare a dataset for training/
-- see above

## 6.) (SUPERVISED) What are some challenges when it comes to collecting labels?
Two common types:
- Logged labels - clicks, transactions, or other interactions. Can be subject to biases - positional or presentation bias. Labels may not accuractly reflect true preferences.
- Human annotated labels - less prone to some bias, but mislabeling is a problem. due ot errors, misunderstanding, or misalignment of incentives (e.g. mechanical turk)

Ways to improve human labeling are:
- clear inscturions
- screening
- tracking (use golden data - pre labeled benchmark data)
- voting (use mulitple respondse)
- validation - expert validator
- captchas - veryif annotatoer is paying attention
- pattern checking (statiscial analysis

## 7.) (UNSUPERVISED) How do you prepare data for clustering algorithms?

- Does not require data labels, but need to preprocess data well

### Categorical Features
- Do not natively accept categorical features (but some do, like deep embedding clusters)
- One hot encoding! But can lead to a high dimensional feature space
- label encoding - assign integer to each featuere
- ordinal encoding - can lead to issues if there is no actual order


### Numerical Features
- log transforms
- Z score scaling (normalization)
- bucketing (i.e. quantile transforms
- missing features:
  - remove samples
  - remove feature
  - impute missing feature values

# 3:) Feature Engineering



## 8.) How do you perform feature engineering for supervised classification or regression?

- in rec sys, features might be demographics, interests, or past behaviors
- for detecting bot traffic, IP addresses, time structure, temporal patterns
- in NLP, features may be n-grams, stems, lemmas, word embeddings, etc
- in CV, features are raw pixel values,
- can be static or dynamic

Contextual features:
- in NLP, contextual features might refer to teh surrounding text of a document
- in Rec Sys, might inluide info about the origination request
- in pricing models, might include nearby listings, number of videws, etc

Sequential features:
- historical data
- past interactions
- in NLP, text is inherently sequential

Crossing features; interaction based crossing features - users x authors / advertisers
Engineering cross is when features are combined
can embed featers (DeepFM)

Derived features:
derived from using existing features. One example is binarization - for exampe (isUpper, IsAcronym, etc
windoed/ decayed mechanisms can be helpful. Radios can be helpful.

## 9.) What are featueres used by generative models?
Generally just take the raw data as input, undergoes some transformation.
- tokenized text aas in put in LMs
- Machine translation - sentence pairs
- OCR - vast collection of scanned images
, need ground routh. Skew normalizntion, etc
- in speech rcogniztion, Mel Frequency Cepstral Coefficeints, derived from filter banks. Filter banks are used to decompose audio signal into frequency components, while MFCCs are obtained by applying additional transofrmation on top of the filter bank outputs.
- in Text to Speech systems, use raw text as input. 

# 4:) Modeling & Evaluation

## 10.) Build a model that performs Information Extraction (i.e. from financial reports)

- Rule based systems, handcrafted by domain experts
- Rule based hybrid approaches
- Statistical models - named entity recognition, hidden marvkov models, etc.
- - Hidden markov models model the probabiilty of a sequence of tages given a sequence of input tokens using joint probability
  - Maximum Entropy Markove Models are extenions that allow for conditional modeling
  - Conditional Random Fields are extensions to MEMMs that solve label bias problems,
- Neural Models: CNNs, BiLSTM
- - CNNs - used for IE by convulting the text using a slideing window, eg. 3 otkens, to extract hte most relevant features, fed to a fully connected layer
  - Bidrirectional LSTM:
- Popular option is BiLSTM-CNN-CRF - combines all three fields

## 11.) What are some evaluation metrics for information extraction?

- Precision - proportion of instances where predicted model is correct
- Recall - proportion of instances that are correct
- F1 score.
- Can also use accuracy

## 12.) Build a model that performs classification (e.g. sentifment analysis)

- rule ased models: quick but not complex
- logistic regression. Estimates probability of events based on values
- Gradient boosting
- Deep Neural Networks

- Factorization machines enable feature crossings, which can help caputre complex interactions.
- Atteniton-based mechanisms, such as transfomer encoders, enable for long term depencendy captures.
- Convvolutions are used for processing spactial data. Useful for image classificaiton, graph rep. and caputing speech signals.
- SVMs not super popular anymore because they don't do as well

## old.) How do you evaulate a classifier model? 
precision, recall, confusion matrix

## 13.) Build a model that performs regression (e.g. demand forecasting)

- linear regression. Good starting point.
- Polynomial regression. Also fine.
- Support Vector Regression. Commonly use d in pricing.
- Tree based approaches. Random Forest Regression; bagging approach that trains each DT on random subsets of the data. BTregression is a boosting approach that trains DTs sequentially
- DNNs - gained immense popularity; caputres higehr oder interactions
- Factorization machines can also be used.

## old.) How do you evaluate quality of a regression model?

MAE, RMSE, MSE, etc


## 14.) What are some techniques for matching identified topics with a taxonomy?

- Top-down approach; use a set of proor words or phrases that are known to be relevant

- Supervissed LDA

- Labeled LDA

- Bottom-up approach - topics are first identified through topic modeling, then extracted

- human in the loop - involve human experts


## 15.) How do you evaluate the performance of a topic modeling algorithm?

- Not universally agreed upon.
- Perplexity - log-likelihood of a test set
- Topic coherence
- Topic diversity
- downstream performance
  

## 16.) Build a model to perform clustering (e.g. group similar news articles)
Document clustering aims to group similar documents together based on content. 

### Document Representation:
- Earliest methods used bag-of-words; each docuemnt represented as a vector of wood frequencies
- - this fails to capture the semantic menaing of words
  - To address this, Term Frequncy Inverse Docuemnt Frequncy was intorduced. FT-IDF takes into account hte importance of each word itn eh document across the corpus, resulting in etter clustering performance
- Word embeddings, such as Word2Vec and GloVe, became popular. Word embeddings encode words into dense continous vectors; capturing the sematntic meaning of words.
- After word embedding, they can be fed into a RNN or LSTM, which can caputre sequential dependenceis. The final state can be used as a fixed length represnetation of the document
- Attention based / transformer models are popular now, caputre long range dependenceis. BERT can handle otu of vocabulary words through subweord tokenization. ThuS BERT represents OOV word as a combination of wubword units


### Cluster Assignment
- k-means was earliest, but is limited in isensitivity to initiazliation, and inability to handle non-spehrical clusters
- hierarchial clutering introducted; groups recurivsly in tree-like strucres
- Density based clustering; identifes clusters based on regions of high density in the data space.
- Gaussian Mxiture Modelsare are probablistic models that aim to find a mixture of multiple Gaussian distributisons that best fit a given dataset.
- Deep Embedding Clustering casts the problem in the form of an autoencoder., uses K-means and KL-divergence loss


## 17.) What are metrics for evaluating the performace of clustering algorithms?

### Unsupervised Clustering Accuracy
- Widely used equivalent of classification accuracy. Measures percentage of documentst aht are correctly assigned.
- Adjusted Mutual Information. takes into account chance clusterings
- Adjusted Rand Index - similarity between two clusterings - considers all pairs and tallies the ones assigned to the same lcuster.
- AMI and ARI are best used depending on the distribution of data (balanced vs unbalanced)
- 

## 18.) Build a model that can generate text (e.g. to summarize a document)

- Noisy Channl  appraoches, inspired by Statisical Machine Translation
- Abstractive summarization is better now.
- RNNs are used for this
- LSTMs are better than RNNs by incorporating memory cells, which have gates
- CNNs were less frequenctly used, but have some adanated s t- they generate fixed size contexutal representaitons while RNNs store informaiton fromt eh entire past. CNNs thus have pore precise control over range dependenceis.
- LLMs now do this best
- Transfomers:
    - T5 (Text To Text Transfmore Transformer) - pre training
    - BART (bidrectional and autoregressiv etransformer, from FAIR. Denosing autoencoder
    - PEGASUS. Random selection.


## 19.) What are some metrics used for assesing quality in generative tasks

Evaluation metrics for generative tasks typically have an objective of measuring similarity between machine-generated text and reference text.
### Co-occurence-based
text summarization primarily uses ROUGE (recall-oriented-understudy for gisting evaluation). ROUGE mesuares the overlap between the generated summary and the reference summary in terms of n-gram co-occurance. 
Machine translation uses BLEU score, typically. 
### Edit distance-based
ASR systems are evaulated using Word Error Rate. WER mesurees difference between machine generated scrpit and reference script
OCR systems evaluate word accuracy and character accuracy. 

# 5:) Deployment & Serving



## 20.) What are steps to prepare a model to serve predictions?

- Data preparation, analysis, and transformation. understand pattersn / relationships that could be leveraged to develop the model
- training tuning and evaluating
- deploying to production - model blessing step - model is validated on a predefined stet of critereia
- upload to model repository
- infrastructure check



## old.) How do you enable the model to serve real-time requests?
-- see above


## 21.) How do you make offline predictions, such as inferring topics from a collection of text?
COntext - offline prodiction - ML models in batch scoring jobs for a large number of data points, not requried in real time.
- eamil and push notifications,
- building datasets that feed into other systems
- Data analysis - sentiment analysis
- demand and prcie models
Approach: unlike online prediciton, batch predition involves the coillection of data in a data warehouse or cloud storage, then process
- cloud storage, suhc as GCS
- data processing pipeline can be used
- batch prediction job can ber used
- Use Hadoop Map Reduce
- write back to cloud storage in CSV or parquet.
- Export to data warehouse or other palce.

Techniques for hyperparameter tuning

## CI/CD

In [ ]:
NLP, information retrieval, knowledge graphs, and knowledge graph embeddings


# Why Grid Search for Hyperparameter Tuning is Often Not Ideal

> **It is generally better *not* to tune hyperparameters strictly on a grid.**

### Key Reasons:

- **Grid search is inefficient in high dimensions.**  
  - If you grid-search 3 values per hyperparameter and have 5 hyperparameters, you need \(3^5 = 243\) evaluations.
  - Most of those evaluations are wasted because only a few hyperparameters matter a lot.
  - Grid search spends time trying many combinations that don't help.

- **Hyperparameter importance is uneven.**  
  - Some hyperparameters (like learning rate) are extremely sensitive.
  - Others (like batch size) are less sensitive.
  - A grid treats all hyperparameters as equally important, which is wrong.

- **Random search and smarter methods are more efficient.**  
  - **Random search** often finds better hyperparameters faster by covering the space better.
  - **Bayesian optimization** intelligently models which areas are promising and focuses search there.
  - Methods like **Hyperband** and **Successive Halving** can early-stop bad candidates to save time.

### Quick Interview Summary:

- Grid search **scales badly** (exponentially) with more hyperparameters.
- **Random search** is usually better.
- **Bayesian and adaptive methods** (e.g., Hyperband) are even better in many cases.

### Bonus Tip:

> Grid search is still acceptable when:
> - There are very few hyperparameters (e.g., 1–2).
> - You have a **large** compute budget.
> - You use a **coarse-to-fine** search strategy: start coarse, refine promising areas with random or adaptive search.
